In [81]:
# Le Duong
# CECS 456 Final Project - Natural Images with 8 classes
# Prof. Zheng
# 5/15/2024

import os
import zipfile
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt

In [82]:
# Loading Dataset

# download and extract dataset
!pip install kaggle
!kaggle datasets download prasunroy/natural-images
with zipfile.ZipFile('natural-images.zip', 'r') as zip_ref:
    zip_ref.extractall('natural-images')

# load dataset
data_dir = 'natural-images/natural_images'  # Directory containing the dataset
classes = os.listdir(data_dir) # different classifications in dataset (8 total)

# initialize empty lists to store images and labels
images = []
labels = []

# iterate through each class directory
for i, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    # iterate through each image in the class directory
    for image_name in os.listdir(class_dir):
        # load image
        image_path = os.path.join(class_dir, image_name)
        image = Image.open(image_path)
        # resize and normalize the image so its consistent
        image = image.resize((28, 28))
        image = np.array(image) / 255.0
        # append the image and its corresponding label to the lists
        images.append(image)
        labels.append(i)

images = np.array(images)
labels = np.array(labels)

# split  dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

#  split the training set further into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
natural-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [83]:
# convert target labels to one-hot encoded vectors (so that target label shape matches model output shape)
y_train= tf.keras.utils.to_categorical(y_train, num_classes=len(classes))
y_valid= tf.keras.utils.to_categorical(y_valid, num_classes=len(classes))
y_test= tf.keras.utils.to_categorical(y_test, num_classes=len(classes))

In [84]:
# Build CNN Model

# initialize the CNN model
cnn = tf.keras.models.Sequential()

# convolutional layers
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding="same", input_shape=[28, 28, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"))
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# flatten layer
cnn.add(tf.keras.layers.Flatten())

# dense layers with L2 regularization
cnn.add(tf.keras.layers.Dense(units=128, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=64, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)))
cnn.add(tf.keras.layers.Dropout(0.5))

# output layer
cnn.add(tf.keras.layers.Dense(units=len(classes), activation="softmax"))

# display model summary
cnn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_70 (Conv2D)          (None, 28, 28, 64)        1792      
                                                                 
 max_pooling2d_42 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_71 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 conv2d_72 (Conv2D)          (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_43 (MaxPooli  (None, 7, 7, 128)         0         
 ng2D)                                                           
                                                                 
 conv2d_73 (Conv2D)          (None, 7, 7, 256)       

In [57]:
# Compile the model
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [60]:
# Train the model
history = cnn.fit(
    X_train,
    y_train,
    batch_size=100,
    epochs=5,
    validation_data=(X_valid, y_valid)
)

Epoch 1/5
45/45 [==============================] - 60s 1s/step - loss: 0.6621 - accuracy: 0.8492 - val_loss: 1.1406 - val_accuracy: 0.7364
Epoch 2/5
45/45 [==============================] - 62s 1s/step - loss: 0.5975 - accuracy: 0.8582 - val_loss: 1.1984 - val_accuracy: 0.6341
Epoch 3/5
45/45 [==============================] - 62s 1s/step - loss: 0.5706 - accuracy: 0.8648 - val_loss: 1.0172 - val_accuracy: 0.6821
Epoch 4/5
45/45 [==============================] - 64s 1s/step - loss: 0.4893 - accuracy: 0.8815 - val_loss: 0.8828 - val_accuracy: 0.7210
Epoch 5/5
45/45 [==============================] - 66s 1s/step - loss: 0.4382 - accuracy: 0.8981 - val_loss: 0.6660 - val_accuracy: 0.8043


In [61]:
# Evaluation of model against test set
score = cnn.evaluate(X_test, y_test)
print('Total loss on Testing Set:', score[0])
print('Accuracy of Testing Set:', score[1])

44/44 [==============================] - 4s 100ms/step - loss: 0.6222 - accuracy: 0.8442
Total loss on Testing Set: 0.6222281455993652
Accuracy of Testing Set: 0.8442028760910034


In [80]:
# Predicting the first three images from the test set
# print the predicted results of the first three images from the test set
# print the real label of the first three images from the test set

# classifications from dataset
class_names = ["airplanes", "car", "cat", "dog", "flower", "fruit", "motorbike", "person"]

# extracting true labels for the first three images from the test set
y_new = np.argmax(y_test[:3], axis=1)

# predicting the first three images from the test set
predictions = cnn.predict(X_test[:3])

# decode the predicted labels
predicted_labels = [class_names[np.argmax(pred)] for pred in predictions]

# print the predicted results
print("Predicted Results:")
for i, label in enumerate(predicted_labels):
    print(f"Image {i+1}: {label}")

# print the real labels of the first three images from the test set
print("\nReal Labels:")
for i in range(3):
    print(f"Image {i+1}: {class_names[y_new[i]]}")

1/1 [==============================] - 0s 55ms/step
Predicted Results:
Image 1: airplanes
Image 2: cat
Image 3: flower

Real Labels:
Image 1: airplanes
Image 2: cat
Image 3: flower
